[Fake News](https://www.kaggle.com/shahidkhan1/fake-news-detection-using-lstm-90-accuracy/data)

 **train.csv**: A full training dataset with the following attributes:


*   id: unique id for a news article
*  title: the title of a news article
* author: author of the news article
* text: the text of the article; could be incomplete
* label: a label that marks the article as potentially unreliable

>> 1: unreliable

>> 0: reliable

**test.csv**: A testing training dataset with all the same attributes at train.csv without the label.

**submit.csv**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/ciclo2/BigData/proyecto-grupal"

In [3]:
!python -m pip install 'fsspec>=0.3.3' #

     |████████████████████████████████| 132 kB 7.4 MB/s 


In [11]:
!kaggle competitions download -c fake-news

 53% 5.00M/9.42M [00:00<00:00, 29.6MB/s]
100% 9.42M/9.42M [00:00<00:00, 35.6MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 16.6MB/s]
 84% 31.0M/37.0M [00:01<00:00, 28.1MB/s]
100% 37.0M/37.0M [00:01<00:00, 32.4MB/s]


In [12]:
! unzip test.csv.zip
! unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [13]:
import pandas as pd
import psutil
import time

In [14]:
df_submit = pd.read_csv("/content/submit.csv")
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")


## Submit

In [ ]:
df_submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [15]:
print("Cantidad de elementos SUBMIT:",len(df_submit.index))

Cantidad de elementos SUBMIT: 5200


## Train

In [16]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [17]:
print("Cantidad de elementos TRAIN:",len(df_train.index))

Cantidad de elementos TRAIN: 20800


In [18]:
display(type(df_train))
display((df_train.count(), len(df_train.columns)))

pandas.core.frame.DataFrame

(id        20800
 title     20242
 author    18843
 text      20761
 label     20800
 dtype: int64, 5)

In [19]:
df_train = df_train[df_train.text.notna()]

## Test

In [20]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
print("Cantidad de elementos TEST:",len(df_test.index))

Cantidad de elementos TEST: 5200


In [21]:
display(type(df_test))
display((df_test.count(), len(df_test.columns)))

pandas.core.frame.DataFrame

(id        5200
 title     5078
 author    4697
 text      5193
 dtype: int64, 4)

In [ ]:
df_test = df_test[df_test.text.notna()]


## DASK: lecturas

In [22]:
import dask.dataframe as dd #

### Submit


In [23]:
%%time
start = time.time()
ddf_submit = dd.from_pandas(df_submit, npartitions = 2) 
ddf_submit.compute()
#print(psutil.Process().memory_info().rss / (1024 * 1024))



CPU times: user 6.57 ms, sys: 1.69 ms, total: 8.26 ms
Wall time: 15 ms


### Train

In [24]:
%%time
ddf_train = dd.from_pandas(df_train, npartitions = 2)
ddf_train.compute()
#print(psutil.Process().memory_info().rss / (1024 * 1024))


CPU times: user 573 ms, sys: 184 ms, total: 757 ms
Wall time: 1.23 s


In [ ]:
len(ddf_train.index)



20800

### Test

In [25]:
%%time
ddf_test = dd.from_pandas(df_test, npartitions = 2)
ddf_test.compute()
#print(psutil.Process().memory_info().rss / (1024 * 1024))


CPU times: user 161 ms, sys: 172 µs, total: 161 ms
Wall time: 305 ms


## DASK: Union Test y Submit

In [26]:
ddf_submit.compute()


,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [27]:
#type(ddf_submit)
#type(ddf_test_not_null)
#ddf_test_merge_submit = ddf_submit.merge(ddf_test, on="id")
%%time
ddf_test_merge_submit =  ddf_test.merge(ddf_submit, on="id")
print(psutil.Process().memory_info().rss / (1024 * 1024))


681.9609375
CPU times: user 28.6 ms, sys: 14.9 ms, total: 43.5 ms
Wall time: 58.6 ms


In [28]:
!pip install partd #

In [29]:
ddf_test_merge_submit.compute()

,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,1
2,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1
3,20811,“Working Class Hero” by John Brennon,Doug Diamond,"Source: CNBC, article by Robert Ferris Arctic ...",1
4,20812,The Rise of Mandatory Vaccinations Means the E...,Shaun Bradley,Written by Shaun Bradley Mandatory vaccinati...,0
...,...,...,...,...,...
2571,25991,E3 Hands-On with ’South Park: The Fractured bu...,Lucas Nolan,The title of South Park: The Fractured but Who...,1
2572,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,1
2573,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,0
2574,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...,1


## DASK: remover valores null de train y test

1.   Test



In [ ]:
#ddf_test_merge_submit['text'].notnull().compute()

In [30]:
%%time
ddf_test_not_null = ddf_test_merge_submit[ddf_test_merge_submit['text'].notnull()].compute() 
ddf_test_not_null
print(psutil.Process().memory_info().rss / (1024 * 1024))


689.16796875
CPU times: user 168 ms, sys: 24.8 ms, total: 192 ms
Wall time: 184 ms


In [31]:
 print("Datos originales:", len(ddf_test), len(ddf_test.columns))
 print("Datos ddf_test_not_null:", len(ddf_test_not_null), len(ddf_test_not_null.columns))

Datos originales: 5200 4
Datos ddf_test_not_null: 5193 5


In [32]:
ddf_test_not_null = ddf_test_not_null.reset_index().drop_duplicates(subset='index', keep='last').set_index('index').sort_index()


In [33]:
ddf_test_not_null.index.duplicated()

array([False, False, False, ..., False, False, False])

1.  Train

In [ ]:
#ddf_train['text'].notnull().compute()

In [34]:
%%time
ddf_train_not_null = ddf_train[ddf_train['text'].notnull()].compute() 
ddf_train_not_null
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

memoria: 689.3828125
CPU times: user 26 ms, sys: 14.4 ms, total: 40.4 ms
Wall time: 49.4 ms


In [35]:
 print("Datos originales:", len(ddf_train), len(ddf_train.columns))
 print("Datos ddf_train_not_null:", len(ddf_train_not_null), len(ddf_train_not_null.columns))

Datos originales: 20761 5
Datos ddf_train_not_null: 20761 5


## DASK: pre procesamiento

In [36]:
ddf_test_not_null["text"]

index
0       Videos #NoDAPL: Native American Leaders Vow to...
1       If at first you don’t succeed, try a different...
2       Trump is USA's antique hero. Clinton will be n...
3       Sunday on NBC’s “Meet the Press,” House Minori...
4       You are here: Home / *Articles of the Bound* /...
                              ...                        
2619    Dr. Caleb Alexander knows how easily older peo...
2620    Earth To Ammosexuals: NRA Admits No One Is Com...
2621    Public concern about faulty automobile airbags...
2622    Pres. Trump on if “tapes” exist of his convers...
2623    Of all the dysfunctions that plague the world’...
Name: text, Length: 2624, dtype: object

In [37]:
ddf_test_not_null["text"]
#ddf_test_not_null

index
0       Videos #NoDAPL: Native American Leaders Vow to...
1       If at first you don’t succeed, try a different...
2       Trump is USA's antique hero. Clinton will be n...
3       Sunday on NBC’s “Meet the Press,” House Minori...
4       You are here: Home / *Articles of the Bound* /...
                              ...                        
2619    Dr. Caleb Alexander knows how easily older peo...
2620    Earth To Ammosexuals: NRA Admits No One Is Com...
2621    Public concern about faulty automobile airbags...
2622    Pres. Trump on if “tapes” exist of his convers...
2623    Of all the dysfunctions that plague the world’...
Name: text, Length: 2624, dtype: object

In [38]:
ddf_train_not_null["text"]

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20761, dtype: object

In [39]:
pip install emoji

     |████████████████████████████████| 170 kB 7.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=d43fd1d4bc9405472a3b4305a800d695eeb087928be467bba116bdab6b16a2b6
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [40]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [43]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'o', "wouldn't", 'whom', 'did', 'or', "aren't", 'a', "won't", 'was', 'then', 'below', 'who', 'do', 'in', 'theirs', 'that', 'their', 'had', "shouldn't", 'yourselves', "mustn't", "doesn't", 'of', 'against', "you'll", 'own', 'and', 'couldn', 'yourself', "you'd", 'shan', 'hers', 'this', 'has', 'over', 'through', 'm', 'most', 'more', 'hadn', 'down', "haven't", 'his', 'haven', 'those', 'which', 'are', 'myself', 'again', 'd', 'ours', 'these', 'with', 'than', 've', 'were', 'we', 'but', 'them', 'before', 'hasn', 't', 'shouldn', 'been', 'will', 'wouldn', 'is', "shan't", 'out', "don't", "hasn't", 'between', 'all', 'itself', "weren't", 'll', 's', 'y', 'why', 'such', 'won', 'isn', 'being', 'at', 'weren', 'ain', 'aren', 'so', "it's", 'its', 'don', 'him', 'having', 'under', 'they', 'ourselves', 'as', "should've", 'our', 're', 'few', "wasn't", 'doing', 'now', 'where', 'it', 'if', 'into', 'she', 'does', 'should', 'when', 'any', 'some', 'your', 'while', 'after', "she's", "isn't", "couldn't", 'once', 'd

In [45]:
import emoji
def give_emoji_free_text(text): 
    return emoji.get_emoji_regexp().sub(r'', text)

In [46]:
def preprocess_new_text(tweet):
  emoticon_string = r"""
  (?:
    [<>]?
    [:;=8]                     
    [\-o\*\']?                 
    [\)\]\(\[dDpP/\:\}\{@\|\\]       
    |
    [\)\]\(\[dDpP/\:\}\{@\|\\]
    [\-o\*\']?                 
    [:;=8]                     
    [<>]?
  )"""

  replacements = (
    ("á", "a"),
    ("é", "e"),
    ("í", "i"),
    ("ó", "o"),
    ("ú", "u"),
  )

  tweet = str(tweet)
  tweet = tweet.lower()
  # Remove urls
  tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
  # Remove user @ references and '#' from tweet
  tweet = re.sub(r'\@\w+|\#','', tweet)
  # Remove user ¿ and ¡ from tweet
  tweet = re.sub(r'\¿|\¡|\?|\!','', tweet)
  # Remove punctuations
  tweet = tweet.translate(str.maketrans('', '', string.punctuation))
  # remove graphical emoji
  tweet = give_emoji_free_text(tweet) 
  # remove textual emoji
  tweet = re.sub(emoticon_string,'',tweet)
  #remove accent mark
  for a, b in replacements:
    tweet = tweet.replace(a, b)

  # Remove stopwords
  tweet_tokens = word_tokenize(tweet)
  filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
  return " ".join(filtered_words)

In [47]:
%%time
df_train.text = ddf_train_not_null["text"].apply(preprocess_new_text)
print(psutil.Process().memory_info().rss / (1024 * 1024))

813.25
CPU times: user 9min 29s, sys: 1.36 s, total: 9min 30s
Wall time: 9min 45s


In [48]:
df_train["text"]

0        house dem aide ’ even see comey ’ letter jason...
1        ever get feeling life circles roundabout rathe...
2        truth might get fired october 29 2016 tension ...
3        videos 15 civilians killed single us airstrike...
4        print iranian woman sentenced six years prison...
                               ...                        
20795    rapper unloaded black celebrities met donald t...
20796    green bay packers lost washington redskins wee...
20797    macy ’ today grew union several great names am...
20798    nato russia hold parallel exercises balkans 11...
20799    david swanson author activist journalist radio...
Name: text, Length: 20761, dtype: object

In [49]:
%%time
df_test.text = ddf_test_not_null["text"].apply(preprocess_new_text)
print(psutil.Process().memory_info().rss / (1024 * 1024))


831.36328125
CPU times: user 1min 10s, sys: 194 ms, total: 1min 11s
Wall time: 1min 11s


## DASK: modelamiento

In [50]:
import numpy as np

In [51]:
%%time
from nltk.stem import PorterStemmer
import re
import tensorflow
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
embedding_vector_features = 40
voc_size = 5000
sent_length = 20
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None
memoria: 1123.15625
CPU times: user 2.29 s, sys: 216 ms, total: 2.51 s
Wall time: 2.64 s


In [52]:
%%time
voc_size = 5000
df_train_str = df_train["text"].astype(str)

onehot_repre = [one_hot(words, voc_size) for words in df_train_str]
onehot_repre[0]
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

memoria: 1486.0625
CPU times: user 10 s, sys: 240 ms, total: 10.3 s
Wall time: 10.3 s


In [ ]:
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sent_length)
print(embedded_docs[0])

[4481 4701 4714 2499 1137  175 2142 2967  815 3188 3065 1859 4910 1191
  624 3524 4606 1831 4580 4910]


In [ ]:
Y = df_train.label  
x_final = np.array(embedded_docs)
y_final = np.array(Y)

In [ ]:
x_final.shape, y_final.shape

((20800, 20), (20800,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [ ]:
%%time
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

Epoch 1/10
218/218 [==============================] - 7s 25ms/step - loss: 0.4070 - accuracy: 0.8065 - val_loss: 0.3072 - val_accuracy: 0.8612
Epoch 2/10
218/218 [==============================] - 5s 24ms/step - loss: 0.2232 - accuracy: 0.9076 - val_loss: 0.2964 - val_accuracy: 0.8728
Epoch 3/10
218/218 [==============================] - 5s 23ms/step - loss: 0.1519 - accuracy: 0.9452 - val_loss: 0.3655 - val_accuracy: 0.8651
Epoch 4/10
218/218 [==============================] - 5s 23ms/step - loss: 0.0970 - accuracy: 0.9663 - val_loss: 0.4491 - val_accuracy: 0.8617
Epoch 5/10
218/218 [==============================] - 5s 23ms/step - loss: 0.0565 - accuracy: 0.9818 - val_loss: 0.5828 - val_accuracy: 0.8530
Epoch 6/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0322 - accuracy: 0.9895 - val_loss: 0.6541 - val_accuracy: 0.8462
Epoch 7/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0196 - accuracy: 0.9937 - val_loss: 0.7498 - val_accuracy: 0.8462

In [ ]:
%%time
classes_x = (model.predict(X_test) > 0.5).astype("int32")
print("memoria:",psutil.Process().memory_info().rss / (1024 * 1024))

memoria: 1744.515625
CPU times: user 1.7 s, sys: 80 ms, total: 1.78 s
Wall time: 1.29 s


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes_x)

array([[2990,  459],
       [ 594, 2821]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes_x)

0.8465909090909091